In [ ]:
# | default_exp gsc_storage


In [ ]:
# | export
from sqlmodel import Session, select
from seo_rat.models import GSCAnalytics
from datetime import datetime,timedelta
from seo_rat.gsc_client import GSCAuth
from functools import partial
from fastcore.basics import compose
from sqlalchemy import func, not_, or_


In [ ]:
# | export
def filter_site(query, site_url: str):
    return query.where(GSCAnalytics.site_url == site_url)


def filter_dates(query, start: str, end: str):
    return query.where(GSCAnalytics.date.between(start, end))


In [ ]:
# | export
def filter_dimension(query, dimension: str, value: str):
    return query.where(getattr(GSCAnalytics, dimension) == value)


In [ ]:
# | export
from sqlmodel import SQLModel


class AnalyticsSummary(SQLModel):
    query: str
    clicks: int
    impressions: int

In [ ]:
#| export 
from urllib.parse import unquote

def normalize_url(url: str) -> str:
    """Normalize URL by decoding percent-encoding and standardizing separators"""
    url = unquote(url)
    url = url.replace("-", " ").replace("_", " ")
    return url


In [ ]:
# | export
def parse_gsc_row(row, site_url, date) -> GSCAnalytics:
    keys = row["keys"]
    fields = ("query", "page", "country", "device")
    dims = dict(zip(fields, keys))
    if dims.get("page"):
        dims["page"] = normalize_url(dims["page"])

    return GSCAnalytics(
        site_url=site_url,
        date=date,
        query=dims.get("query"),
        page=dims.get("page"),
        country=dims.get("country"),
        device=dims.get("device"),
        clicks=row["clicks"],
        impressions=row["impressions"],
        ctr=row["ctr"],
        position=row["position"],
    )

In [ ]:
# | export
def store_gsc_data(session: Session, site_url: str, date: str, rows: list[dict]):
    """Store GSC data with upsert (update or insert)"""
    for row in rows:
        record = parse_gsc_row(row, site_url, date)
        existing = session.exec(
            select(GSCAnalytics).where(
                GSCAnalytics.site_url == record.site_url,
                GSCAnalytics.date == record.date,
                GSCAnalytics.query == record.query,
                GSCAnalytics.page == record.page,
                GSCAnalytics.country == record.country,
                GSCAnalytics.device == record.device,
            )
        ).first()
        if existing:
            record.id = existing.id
        session.merge(record)
    session.commit()


In [ ]:
# | hide
#| eval: false
from fastcore.test import test_eq
from seo_rat.gsc_client import GSCAuth, fetch_gsc_data, get_date_range
from seo_rat.sqlite_db import SQLiteDB
from sqlmodel import Session
from pprint import pprint


auth = GSCAuth()
start, end = get_date_range("last_days", days=30)
data = fetch_gsc_data(auth, "sc-domain:kareemai.com", start, end)


In [ ]:
# | hide
#| eval: false
db = SQLiteDB()
with db.get_session() as session:
    store_gsc_data(session, "sc-domain:kareemai.com", start, data[:5])
    # verfiy stored
    test_eq(len(session.exec(select(GSCAnalytics)).all()) > 4, True)

In [ ]:
# | export
def get_top_queries(
    session: Session,
    site_url: str,
    start_date: str,
    end_date: str,
    country: str | None = None,
    page_path: str | None = None,
    limit: int = 10,
) -> list[dict]:
    """Get top performing queries, optionally filtered by page"""
    base_query = select(
        GSCAnalytics.query,
        func.sum(GSCAnalytics.clicks).label("total_clicks"),
        func.sum(GSCAnalytics.impressions).label("total_impressions"),
        func.avg(GSCAnalytics.position).label("avg_position"),
        func.avg(GSCAnalytics.ctr).label("avg_ctr"),
    )
    filters = [
        partial(filter_site, site_url=site_url),
        partial(filter_dates, start=start_date, end=end_date),
    ]
    if country:
        filters.append(partial(filter_dimension, dimension="country", value=country))
    if page_path:
        filters.append(lambda q: q.where(GSCAnalytics.page.contains(page_path)))

    query = (compose(*filters)(base_query)
        .where(GSCAnalytics.query.isnot(None))
        .group_by(GSCAnalytics.query)
        .order_by(func.sum(GSCAnalytics.clicks).desc())
        .limit(limit))

    return [row._asdict() for row in session.exec(query)]


In [ ]:
# | hide
#| eval: false

test_top_queries = get_top_queries(
    session,
    site_url="sc-domain:kareemai.com",
    start_date=start,
    end_date=end,
    country="egy",
)

In [ ]:
# | export
def filter_exclude_pages(query, exclude_pages: list[str]):
    filters = [GSCAnalytics.page.contains(page) for page in exclude_pages]
    return query.where(not_(or_(*filters)))


In [ ]:
# | export
def get_top_queries_excluding_pages(
    session: Session,
    site_url: str,
    start_date: str,
    end_date: str,
    exclude_pages: list[str],
    country: str | None = None,
    limit: int = 10,
) -> list[dict]:
    """Get top queries excluding certain pages"""
    base_query = select(
        GSCAnalytics.query,
        func.sum(GSCAnalytics.clicks).label("total_clicks"),
        func.sum(GSCAnalytics.impressions).label("total_impressions"),
    )
    filters = [
        partial(filter_site, site_url=site_url),
        partial(filter_dates, start=start_date, end=end_date),
        partial(filter_exclude_pages, exclude_pages=exclude_pages),
    ]
    if country:
        filters.append(partial(filter_dimension, dimension="country", value=country))

    query = (
        compose(*filters)(base_query)
        .where(GSCAnalytics.query.isnot(None))
        .group_by(GSCAnalytics.query)
        .order_by(func.sum(GSCAnalytics.clicks).desc())
        .limit(limit)
    )

    return [row._asdict() for row in session.exec(query)]


In [ ]:
# | hide
#| eval: false
from sqlalchemy.testing.exclusions import exclude

test_exclude_page_from_top_queries = get_top_queries_excluding_pages(
    session,
    site_url="sc-domain:kareemai.com",
    start_date=start,
    end_date=end,
    exclude_pages=[
        "https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%207i.html",
        "https://kareemai.com/blog/posts/products_reviews/Huawei%20freebuds%205i.html",
    ],
    country="egy",
)
pprint(test_exclude_page_from_top_queries)

[]


In [ ]:
# | export
def get_page_analytics(
    session: Session, site_url: str, page_path: str, start_date: str, end_date: str
) -> dict:
    """Get analytics for a specific page"""
    base_query = select(
        func.sum(GSCAnalytics.clicks).label("total_clicks"),
        func.sum(GSCAnalytics.impressions).label("total_impressions"),
        func.avg(GSCAnalytics.position).label("avg_position"),
        func.avg(GSCAnalytics.ctr).label("avg_ctr"),
    )
    filters = compose(
        partial(filter_site, site_url=site_url),
        partial(filter_dates, start=start_date, end=end_date),
    )
    query = filters(base_query).where(GSCAnalytics.page.contains(page_path))

    row = session.exec(query).first()

    # Top queries need a separate query
    q_query = filters(select(GSCAnalytics.query)).where(
        GSCAnalytics.page.contains(page_path),
        GSCAnalytics.query.isnot(None),
    )
    top_queries = list(set(session.exec(q_query)))[:10]

    return {
        "page_path": page_path,
        "total_clicks": row.total_clicks or 0,
        "total_impressions": row.total_impressions or 0,
        "avg_position": row.avg_position or 0,
        "avg_ctr": row.avg_ctr or 0,
        "top_queries": top_queries,
    }


In [ ]:
# | hide
#| eval: false
test_page_analytics = get_page_analytics(
    session,
    site_url="sc-domain:kareemai.com",
    page_path="https://kareemai.com",
    start_date=start,
    end_date=end,
)
pprint(test_page_analytics)

{'avg_ctr': 0.1507487922705314,
 'avg_position': 3.977165861513688,
 'page_path': 'https://kareemai.com',
 'top_queries': ['huawei freebuds 7i vs 5i',
                 'huawei freebuds 7i',
                 'huawei freebuds 7i review',
                 'freebuds 7i',
                 'sample query'],
 'total_clicks': 21,
 'total_impressions': 551}


In [ ]:
# | export
def get_analytics_by_date_range(
    session: Session, site_url: str, start_date: str, end_date: str
) -> list[GSCAnalytics]:
    """Get all analytics for date range"""
    base_query = select(GSCAnalytics)
    build_query = compose(
        partial(filter_site, site_url=site_url),
        partial(filter_dates, start=start_date, end=end_date),
    )
    query = build_query(base_query)

    return session.exec(query).all()


In [ ]:
# | hide
#| eval: false

test_get_analytics_by_date_range = get_analytics_by_date_range(
    session, site_url="sc-domain:kareemai.com", start_date=start, end_date=end
)
pprint(test_get_analytics_by_date_range)

[GSCAnalytics(id=6, site_url='sc-domain:kareemai.com', query='freebuds 7i', country='hkg', clicks=2, ctr=0.16666666666666666, created_at=datetime.datetime(2026, 2, 28, 9, 26, 20, 509333), page='https://kareemai.com/blog/posts/products reviews/Huawei freebuds 7i.html', date='2026-01-26', device='MOBILE', impressions=12, position=5.5),
 GSCAnalytics(id=2, site_url='sc-domain:kareemai.com', query='huawei freebuds 7i', country='are', clicks=5, ctr=0.014492753623188406, created_at=datetime.datetime(2026, 2, 28, 9, 26, 20, 490468), page='https://kareemai.com/blog/posts/products reviews/Huawei freebuds 7i.html', date='2026-01-26', device='MOBILE', impressions=345, position=5.498550724637681),
 GSCAnalytics(id=3, site_url='sc-domain:kareemai.com', query='huawei freebuds 7i', country='sgp', clicks=3, ctr=0.04, created_at=datetime.datetime(2026, 2, 28, 9, 26, 20, 499403), page='https://kareemai.com/blog/posts/products reviews/Huawei freebuds 7i.html', date='2026-01-26', device='MOBILE', impressi

In [ ]:
# | export
def get_trends(
    session: Session,
    site_url: str,
    start_date: str,
    end_date: str,
    dimension: str | None = None,
) -> list[dict]:
    """Get trends over time, optionally grouped by dimension"""
    columns = [
        GSCAnalytics.date,
        func.sum(GSCAnalytics.clicks).label("clicks"),
        func.sum(GSCAnalytics.impressions).label("impressions"),
        func.avg(GSCAnalytics.position).label("avg_position"),
        func.avg(GSCAnalytics.ctr).label("avg_ctr"),
    ]
    group_by = [GSCAnalytics.date]

    if dimension:
        dim_col = getattr(GSCAnalytics, dimension)
        columns.insert(1, dim_col)
        group_by.append(dim_col)

    base_query = select(*columns)
    filters = compose(
        partial(filter_site, site_url=site_url),
        partial(filter_dates, start=start_date, end=end_date),
    )
    query = filters(base_query).group_by(*group_by).order_by(GSCAnalytics.date)

    return [row._asdict() for row in session.exec(query)]


In [ ]:
# | hide
#| eval: false
test_get_trends = get_trends(
    session, site_url="sc-domain:kareemai.com", start_date=start, end_date=end
)
pprint(test_get_trends)

[{'avg_ctr': 0.17089855072463767,
  'avg_position': 4.132599033816425,
  'clicks': 16,
  'date': '2026-01-26',
  'impressions': 451},
 {'avg_ctr': 0.05,
  'avg_position': 3.2,
  'clicks': 5,
  'date': '2026-02-01',
  'impressions': 100}]


In [ ]:
# | export
def get_analytics_by(
    session: Session,
    site_url: str,
    start_date: str,
    end_date: str,
    dimension: str,
    value: str,
) -> list[AnalyticsSummary]:

    base_query = select(
        GSCAnalytics.query,
        func.sum(GSCAnalytics.clicks).label("clicks"),
        func.sum(GSCAnalytics.impressions).label("impressions"),
    )

    build_query = compose(
        partial(filter_site, site_url=site_url),
        partial(filter_dates, start=start_date, end=end_date),
        partial(filter_dimension, dimension=dimension, value=value),
    )
    return [
        AnalyticsSummary(**row._asdict())
        for row in session.exec(build_query(base_query).group_by(GSCAnalytics.query))
    ]


In [ ]:
# | hide
#| eval: false
test_result = get_analytics_by(
    session,
    site_url="sc-domain:kareemai.com",
    start_date=start,
    end_date=end,
    dimension="country",
    value="egy",
)

pprint(test_result)


[]


In [ ]:
# | hide
#| eval: false
test_get_analytics_by_device = get_analytics_by(
    session,
    site_url="sc-domain:kareemai.com",
    start_date=start,
    end_date=end,
    dimension="device",
    value="MOBILE",
)
pprint(test_get_analytics_by_device)


[AnalyticsSummary(query='freebuds 7i', clicks=2, impressions=12),
 AnalyticsSummary(query='huawei freebuds 7i', clicks=8, impressions=420),
 AnalyticsSummary(query='huawei freebuds 7i review', clicks=3, impressions=9),
 AnalyticsSummary(query='huawei freebuds 7i vs 5i', clicks=3, impressions=10),
 AnalyticsSummary(query='sample query', clicks=5, impressions=100)]


In [ ]:
# | export
from seo_rat.gsc_client import fetch_gsc_data
import time

def store_single_date(session: Session, auth: GSCAuth, site_url: str, date: str) -> int:
    """Store GSC data for a single date"""

    rows = fetch_gsc_data(auth, site_url, date, date)
    store_gsc_data(session, site_url, date, rows)
    return len(rows)


In [ ]:
# | hide
# | test
#| eval: false
count = store_single_date(session, auth, "sc-domain:kareemai.com", start)
test_eq(count > 0, True)


In [ ]:
# | test
#| eval: false
# Check what's actually stored
stored = session.exec(select(GSCAnalytics).limit(1)).first()
print(f"Query: {stored.query}")
print(f"Clicks: {stored.clicks}")
print(f"Date: {stored.date}")
test_eq(stored.site_url, "sc-domain:kareemai.com")


Query: sample query
Clicks: 5
Date: 2026-02-01


In [ ]:
# | export
def store_date_range(
    session: Session, auth: GSCAuth, site_url: str, start_date: str, end_date: str
) -> dict:
    """Store GSC data for a date range with progress"""
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    total_days = (end - start).days + 1

    results = {"successful": [], "failed": [], "total_records": 0}

    for day_num in range(total_days):
        date_str = (start + timedelta(days=day_num)).strftime("%Y-%m-%d")
        print(f"Processing {date_str} ({day_num + 1}/{total_days})...")

        try:
            count = store_single_date(session, auth, site_url, date_str)
            results["successful"].append(date_str)
            results["total_records"] += count
        except Exception as e:
            results["failed"].append({"date": date_str, "error": str(e)})

        time.sleep(1)

    return results



In [ ]:
# | test
#| eval: false
result = store_date_range(session, auth, "sc-domain:kareemai.com", start, start)
test_eq(len(result["successful"]), 1)
print(f"Stored {result['total_records']} records")


Processing 2026-01-26 (1/1)...
Stored 259 records


In [ ]:
#| test
#| eval: false
r = session.exec(select(GSCAnalytics).limit(1)).first()
print(f"Query: {r.query}")
print(f"Page: {r.page}")
print(f"Country: {r.country}")
print(f"Device: {r.device}")
print(f"Clicks: {r.clicks}")
print(f"Impressions: {r.impressions}")
print(f"CTR: {r.ctr}")
print(f"Position: {r.position}")


Query: sample query
Page: https://kareemai.com/test page.html
Country: are
Device: MOBILE
Clicks: 5
Impressions: 100
CTR: 0.05
Position: 3.2


In [ ]:
# | test
#| eval: false
top = get_top_queries(session, "sc-domain:kareemai.com", start, start, limit=5)
for q in top:
    print(
        f"{q['query']}: {q['total_clicks']} clicks, {q['total_impressions']} impressions"
    )


huawei freebuds 7i review: 4 clicks, 45 impressions
huawei freebuds 7i: 3 clicks, 191 impressions
freebuds 7i: 3 clicks, 35 impressions
huawei freebuds 7i wireless earbuds: 1 clicks, 5 impressions
huawei freebuds 7i vs 5i: 1 clicks, 5 impressions


In [ ]:
from seo_rat.sqlite_db import SQLiteDB
from seo_rat.models import GSCAnalytics

In [ ]:
from sqlalchemy import delete


In [ ]:
# | hide
#| eval: false

db = SQLiteDB()
session = db.get_session()

session.exec(delete(GSCAnalytics))
session.commit()


In [ ]:
# | hide
#| eval: false

# delay 3 because GSC api limitation
end = datetime.now() - timedelta(days=3)
start = end - timedelta(days=365 * 3)

result = store_date_range(
    session,
    auth,
    "sc-domain:kareemai.com",
    start.strftime("%Y-%m-%d"),
    end.strftime("%Y-%m-%d"),
)


Processing 2023-02-26 (1/1096)...
Processing 2023-02-27 (2/1096)...
Processing 2023-02-28 (3/1096)...
Processing 2023-03-01 (4/1096)...
Processing 2023-03-02 (5/1096)...
Processing 2023-03-03 (6/1096)...
Processing 2023-03-04 (7/1096)...
Processing 2023-03-05 (8/1096)...
Processing 2023-03-06 (9/1096)...
Processing 2023-03-07 (10/1096)...
Processing 2023-03-08 (11/1096)...
Processing 2023-03-09 (12/1096)...
Processing 2023-03-10 (13/1096)...
Processing 2023-03-11 (14/1096)...
Processing 2023-03-12 (15/1096)...
Processing 2023-03-13 (16/1096)...
Processing 2023-03-14 (17/1096)...
Processing 2023-03-15 (18/1096)...
Processing 2023-03-16 (19/1096)...
Processing 2023-03-17 (20/1096)...
Processing 2023-03-18 (21/1096)...
Processing 2023-03-19 (22/1096)...
Processing 2023-03-20 (23/1096)...
Processing 2023-03-21 (24/1096)...
Processing 2023-03-22 (25/1096)...
Processing 2023-03-23 (26/1096)...
Processing 2023-03-24 (27/1096)...
Processing 2023-03-25 (28/1096)...
Processing 2023-03-26 (29/109